In [2]:
import numpy as np
import matplotlib.pyplot as plt
import GPy

### Function Definitions

In [3]:
def f_H(x):
    return(6.0*x-2.0)**2 * np.sin(12.0*x-4.0)

def f_L(x):
    return 0.5*f_H(x) + 10.0*(x-0.5) - 5.0

def Normalize(X, X_m, X_s):
    return (X-X_m) / (X_s)

In [4]:
N_H = 3
N_L = 8
D   = 1

lb = 0.0
ub = 1.0
noise_L = 0.00
noise_H = 0.00

# Training data
X_L = np.random.uniform(lb,ub, size = (N_L,1))
y_L = f_L(X_L) + noise_L*np.random.randn(N_L,D)

X_H = np.random.uniform(lb,ub, size = (N_H,1))
y_H = f_H(X_H) + noise_H*np.random.randn(N_H,D)

In [8]:
kernel1 = GPy.kern.RBF(1)

# Train the low fidelity
m1 = GPy.models.GPRegression(X=X_L, Y=y_L, kernel=kernel1)

m1[".*Gaussian_noise"] = m1.Y.var()*0.01
m1[".*Gaussian_noise"].fix()

m1.optimize(max_iters = 500)

m1[".*Gaussian_noise"].unfix()
m1[".*Gaussian_noise"].constrain_positive()

# Restart optimization to find the global minimum instead of a local one
m1.optimize_restarts(20, optimizer = "bfgs",  max_iters = 1000)

mu1, v1 = m1.predict(X_H)

reconstraining parameters GP_regression.Gaussian_noise.variance


Optimization restart 1/20, f = 11.529961940634857
Optimization restart 2/20, f = 18.325776776243302
Optimization restart 3/20, f = 18.32577677479371
Optimization restart 4/20, f = 18.3257767748015
Optimization restart 5/20, f = 18.32577677478043
Optimization restart 6/20, f = 18.325776775353166
Optimization restart 7/20, f = 18.325776774783257
Optimization restart 8/20, f = 18.3257767747815
Optimization restart 9/20, f = 18.32577677482623
Optimization restart 10/20, f = 18.3257767751417
Optimization restart 11/20, f = 18.325776774858365
Optimization restart 12/20, f = 18.325776774788036
Optimization restart 13/20, f = 18.325776774782405
Optimization restart 14/20, f = 18.325776774780046
Optimization restart 15/20, f = 18.325776774780795
Optimization restart 16/20, f = 18.325776774800325
Optimization restart 17/20, f = 18.325776774779442
Optimization restart 18/20, f = 18.32577677551057
Optimization restart 19/20, f = 18.325776774833365
Optimization restart 20/20, f = 18.325776775233045

In [10]:
# Train the higher fidelity
kernel2 = GPy.kern.Bias * GPy.kern.Bias * GPy.kern.RBF + GPy.kern.RBF
XX = np.hstack((X_H,mu1))

m2 = GPy.models.GPRegression(X=XX, Y=y_H, kernel=kernel2)

m2[".*Gaussian_noise"] = m2.Y.var()*0.01
m2[".*Gaussian_noise"].fix()

m2.optimize(max_iters = 500)

m2[".*Gaussian_noise"].unfix()
m2[".*Gaussian_noise"].constrain_positive()

# Restart optimization to find the global minimum instead of a local one
m2.optimize_restarts(20, optimizer = "bfgs",  max_iters = 1000)

TypeError: unsupported operand type(s) for *: 'KernCallsViaSlicerMeta' and 'KernCallsViaSlicerMeta'